# Create dbGaP spreadsheet - CalTech
## Create folder and copy definitions

In [1]:
import pandas as pd
from shutil import copytree
from shutil import rmtree
import pathlib
import json
import yaml
import hubmapbags
import pickle
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from warnings import warn as warning
from datetime import datetime
import warnings

instance = 'prod'
token = ''

In [2]:
now = datetime.now()
output_directory = 'data'
if not Path(output_directory).exists():
    Path(output_directory).mkdir()

report_output_directory = 'uuid-protected-data-report'
if not Path(report_output_directory).exists():
    Path(report_output_directory).mkdir()

report_output_filename = report_output_directory + '/' + str(now.strftime('%Y%m%d')) + '.pkl'
print('File found on disk. Loading ' + report_output_filename + '.')
report = pd.read_pickle(report_output_filename.replace('tsv', 'pkl'))

File found on disk. Loading uuid-protected-data-report/20230120.pkl.


In [3]:
def get_dbgap_study_id( datum ):
        if ( datum['group_name'] == 'University of California San Diego TMC' ) or \
                ( datum['group_name'] == 'Broad Institute RTI' and datum['data_type'] == 'Slide-seq' ):
                return 'phs002249'
        elif datum['group_name'] == 'Stanford TMC':
                return 'phs002272'
        else:
                return None

In [4]:
report['group_name'].unique()

array(['Stanford TMC', 'University of California San Diego TMC',
       'Broad Institute RTI', 'University of Florida TMC',
       'EXT - Human Cell Atlas', 'TMC - University of Connecticut',
       'California Institute of Technology TMC'], dtype=object)

In [5]:
report = report[report['group_name'] == 'California Institute of Technology TMC']

In [6]:
warnings.filterwarnings('ignore')

if not Path('caltech-dataframe.tsv').exists():
    columns = ['phs_accession','sample_ID','library_ID', \
               'title','group_name','data_type','library_strategy', \
               'library_source','library_selection', \
               'library_layout','platform','instrument_model', \
               'design_description','reference_genome_assembly', \
               'alignment_software']

    df = pd.DataFrame(columns=columns)

    phs_accession = '<foobar>'
    
    rows = []
    lib_counter = 1
    for index, datum in tqdm(report.iterrows()):
        metadata = hubmapbags.apis.get_dataset_info(datum['hubmap_id'], instance='prod', token=token)
        pmetadata = hubmapbags.apis.get_provenance_info(datum['hubmap_id'], instance='prod', token=token)

        organ = pmetadata['organ_type'][0]

        library_ID = metadata['ingest_metadata']['metadata']['library_id']
        library_ID = library_ID + '-' + datum['hubmap_id']

        instrument_model = metadata['ingest_metadata']['metadata']['acquisition_instrument_model']

        platform = metadata['ingest_metadata']['metadata']['acquisition_instrument_vendor']

        if metadata['ingest_metadata']['metadata']['library_layout'] == 'paired-end':
            library_layout = 'paired'
        else:
            library_layout = 'single'

        library_selection = 'other'

        if metadata['ingest_metadata']['metadata']['analyte_class'] == 'DNA':
            library_strategy = 'ATAC-seq'
            title= datum['data_type'] + ' of human ' + organ
            library_source = 'GENOMIC'
        else:
            library_strategy = 'RNA-Seq'
            title= datum['data_type'] + ' of human ' + organ
            library_source = 'TRANSCRIPTOMIC'

        library_construction_protocols_io_doi = metadata['ingest_metadata']['metadata']['library_construction_protocols_io_doi']
        sequencing_reagent_kit = metadata['ingest_metadata']['metadata']['sequencing_reagent_kit']
        design_description = f'<TEMP>SNARE-seq2 was performed as outlined (Nature Protocols, DOI:10.1038/s41596-021-00507-3) and according to the following protocols.io protocol: dx.doi.org/{library_construction_protocols_io_doi}. The library was sequenced on the {platform} {instrument_model} system using the {sequencing_reagent_kit} kit.'

        row = {'phs_accession':phs_accession, \
               'sample_ID':datum['hubmap_id'], \
               'library_ID':library_ID, \
               'title':title, \
               'group_name':datum['group_name'], \
               'data_type':datum['data_type'], \
               'library_strategy':library_strategy , \
               'library_source':library_source , \
               'library_selection':library_selection, \
               'library_layout':library_layout, \
               'platform':platform, \
               'instrument_model':instrument_model, \
               'design_description':design_description, \
               'reference_genome_assembly':None, \
               'alignment_software':None }

        rows.append(row)

    for row in rows:
        df = df.append(row, ignore_index=True)

    df.to_csv( 'caltech-dataframe.tsv', sep='\t', index=False )
else:
    df = pd.read_csv( 'caltech-dataframe.tsv', sep='\t' )

33it [00:11,  2.95it/s]


In [7]:
if Path('caltech.pkl').exists():
    file = open('caltech.pkl', 'rb')
    files = pickle.load(file)
    file.close()

    df["filetype0"]=None
    df["filename0"]=None
    df["checksum0"]=None
    df["filetype1"]=None
    df["filename1"]=None
    df["checksum1"]=None
    df["filetype2"]=None
    df["filename2"]=None
    df["checksum2"]=None
    df["filetype3"]=None
    df["filename3"]=None
    df["checksum3"]=None
    df["filetype4"]=None
    df["filename4"]=None
    df["checksum4"]=None
    df["filetype5"]=None
    df["filename5"]=None
    df["checksum5"]=None
    df["filetype6"]=None
    df["filename6"]=None
    df["checksum6"]=None
    df["filetype7"]=None
    df["filename7"]=None
    df["checksum7"]=None

    for index, datum in df.iterrows():
        if len(files[index]) == 1:
            keys = list(files[index].keys())
            df.loc[index,'checksum0'] = files[index][keys[0]]
            df.loc[index,'filename0'] = keys[0]
            df.loc[index,'filetype0'] = 'fastq'
        elif len(files[index]) == 2:
            keys = list(files[index].keys())
            df.loc[index,'checksum0'] = files[index][keys[0]]
            df.loc[index,'filename0'] = keys[0]
            df.loc[index,'checksum1'] = files[index][keys[1]]
            df.loc[index,'filename1'] = keys[1]
            df.loc[index,'filetype0'] = df.loc[index,'filetype1'] = 'fastq'
        else:
            keys = list(files[index].keys())
            df.loc[index,'checksum0'] = files[index][keys[0]]
            df.loc[index,'filename0'] = keys[0]
            df.loc[index,'checksum1'] = files[index][keys[1]]
            df.loc[index,'filename1'] = keys[1]
            df.loc[index,'checksum2'] = files[index][keys[2]]
            df.loc[index,'filename2'] = keys[2]
            df.loc[index,'checksum3'] = files[index][keys[3]]
            df.loc[index,'filename3'] = keys[3]
            df.loc[index,'checksum4'] = files[index][keys[4]]
            df.loc[index,'filename4'] = keys[4]
            df.loc[index,'checksum5'] = files[index][keys[5]]
            df.loc[index,'filename5'] = keys[5]
            df.loc[index,'checksum6'] = files[index][keys[6]]
            df.loc[index,'filename6'] = keys[6]
            df.loc[index,'checksum7'] = files[index][keys[7]]
            df.loc[index,'filename7'] = keys[7]
            df.loc[index,'filetype0'] = df.loc[index,'filetype1'] = \
            df.loc[index,'filetype2'] = df.loc[index,'filetype3'] = \
            df.loc[index,'filetype4'] =  df.loc[index,'filetype5'] = \
            df.loc[index,'filetype6'] =  df.loc[index,'filetype7'] = 'fastq'

    df.to_csv('caltech-spreadsheet.tsv', sep='\t', index=False)